In [1]:
import pandas as pd
import yaml
import csv
import json
import time

import torch
import torch.nn as nn
from torch import nn, optim, Tensor
import torch.nn.functional as F

import numpy as np
from typing import List, Tuple, Any, Optional, Dict

from data import Instance, Data, Dictionary

In [2]:
Triple = Fact = Tuple[str, str, str]
Story = List[Fact]

In [ ]:
data = Data()
dictionary = Dictionary(data.all_triples, data)

In [4]:
dictionary.num_entities

78

In [5]:
dictionary.entity2idx

{'Aida': 0,
 'Alan': 1,
 'Alfred': 2,
 'Alma': 3,
 'Angela': 4,
 'Anna': 5,
 'Annette': 6,
 'Annie': 7,
 'Anthony': 8,
 'Asia': 9,
 'Beverly': 10,
 'Carlos': 11,
 'Chad': 12,
 'Christopher': 13,
 'Clarence': 14,
 'Connie': 15,
 'Craig': 16,
 'Dan': 17,
 'Darnell': 18,
 'David': 19,
 'Deborah': 20,
 'Donald': 21,
 'Donna': 22,
 'Dorothy': 23,
 'Dwight': 24,
 'Elizabeth': 25,
 'Elliott': 26,
 'Ellis': 27,
 'Emmanuel': 28,
 'Enedina': 29,
 'Eunice': 30,
 'Gabrielle': 31,
 'Guillermina': 32,
 'James': 33,
 'Jason': 34,
 'Jeanna': 35,
 'Jeffrey': 36,
 'Jennifer': 37,
 'Jeremy': 38,
 'Joe': 39,
 'John': 40,
 'Jose': 41,
 'Karen': 42,
 'Kevin': 43,
 'Kirk': 44,
 'Lee': 45,
 'Lewis': 46,
 'Lisa': 47,
 'Lucille': 48,
 'Marci': 49,
 'Marian': 50,
 'Mark': 51,
 'Matthew': 52,
 'May': 53,
 'Michael': 54,
 'Micheal': 55,
 'Michele': 56,
 'Michelle': 57,
 'Molly': 58,
 'Myrna': 59,
 'Paul': 60,
 'Reta': 61,
 'Robert': 62,
 'Rosie': 63,
 'Russell': 64,
 'Sandra': 65,
 'Scott': 66,
 'Sheila': 67,
 'Si

In [4]:
# Aga
# entity_embeddings = nn.Embedding(data.nb_entities, rank, sparse=True)
# predicate_embeddings = nn.Embedding(data.nb_predicates, rank, sparse=True)

In [19]:
data.train[0].target

('Donald', 'aunt', 'Dorothy')

In [20]:
entity_embeddings = nn.Embedding(dictionary.num_entities, 100, sparse=True)
relation_embeddings = nn.Embedding(dictionary.num_predicates, 100, sparse=True)

In [21]:
entity_embeddings.weight.shape

torch.Size([78, 100])

In [9]:
def encode_arguments(facts: List[Fact],
                     entity_embeddings: Tensor,
                     entity_to_idx: Dict[str, int],
                     device: Optional[torch.device] = None) -> Tuple[Tensor, Tensor]:
    indices_np = np.array([[entity_to_idx[s], entity_to_idx[o]] for s, _, o in facts], dtype=np.int64)
    indices = torch.tensor(indices_np, dtype=torch.long, device=device)
    emb = F.embedding(indices, entity_embeddings)
    return emb[:, 0, :], emb[:, 1, :]


def encode_entities(facts: List[Fact],
                    entity_embeddings: Tensor,
                    entity_to_idx: Dict[str, int]) -> Tensor:
    indices_lst = sorted({entity_to_idx[e] for s, r, o in facts for e in {s, o}})
    indices = torch.tensor(indices_lst, dtype=torch.long)
    emb = F.embedding(indices, entity_embeddings)
    return emb

def encode_relation(facts: List[Fact],
                    relation_embeddings: Tensor,
                    relation_to_idx: Dict[str, int],
                    device: Optional[torch.device] = None) -> Tensor:
    indices_np = np.array([relation_to_idx[r] for _, r, _ in facts], dtype=np.int64)
    indices = torch.tensor(indices_np, dtype=torch.long, device=device)
    res = F.embedding(indices, relation_embeddings)
    return res

In [22]:
story, target = data.train[0].story, data.train[0].target
len(story)

2

In [35]:
indices_np = np.array([[dictionary.entity2idx[s], dictionary.entity2idx[o]] for s, _, o in story])
indices = torch.tensor(indices_np, dtype=torch.long)
indices

tensor([[21, 54],
        [54, 23]])

In [11]:
embeddings = encode_entities([story[0]], entity_embeddings.weight, dictionary.entity2idx)
story_arg1, story_arg2 = encode_arguments([story[0]], entity_embeddings.weight, dictionary.entity2idx)

In [12]:
s, r, o = target

In [13]:
target_lst: List[Tuple[str, str, str]] = [(s, x, o) for x in data.relation_lst]

In [14]:
rel_emb = encode_relation(target_lst, relation_embeddings.weight, dictionary.predicate2idx)

In [13]:
story

NameError: name 'story' is not defined

In [23]:
story

[('Donald', 'father', 'Michael'), ('Michael', 'sister', 'Dorothy')]